In [1]:
import pandas as pd
import glob
import os
import lmdb
import pickle
import ast
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import torch
from models import models_vit
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
from util.pos_embed import interpolate_pos_embed

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
v = models_vit.VisionTransformer(18, img_size=(256, 416, 416), num_heads=16, 
                                     segment_length="[2704, 5408, 10816, 21632, 43264]",)
moddel = v.to(device).half()
ct = torch.randn(1, 1, 256, 416, 416, device=device).half()
preds = v(ct)

Number of patches: 43264


In [3]:
preds.shape
preds

torch.Size([1, 18])

tensor([[-0.6489,  0.2847, -0.4468, -0.3201,  0.9175, -0.2333, -0.3855, -0.1503,
          0.0775, -0.2583,  0.0063, -0.2595,  0.2935, -0.8589, -0.7666, -0.1348,
         -0.4915, -0.0210]], device='cuda:2', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)

In [2]:
finetune = 'checkpoints/checkpoint-799.pth'

checkpoint = torch.load(finetune, map_location='cpu')

print("Load pre-trained checkpoint from: %s" % finetune)
checkpoint_model = checkpoint['model']
model = models_vit.__dict__['vit_base_patch16'](
        num_classes=18,
    )
state_dict = model.state_dict()
for k in ['head.weight', 'head.bias']:
    if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
        print(f"Removing key {k} from pretrained checkpoint")
        del checkpoint_model[k]

# interpolate position embedding
interpolate_pos_embed(model, checkpoint_model)

# load pre-trained model
msg = model.load_state_dict(checkpoint_model, strict=False)
print(msg)

Load pre-trained checkpoint from: checkpoints/checkpoint-799.pth
Number of patches: 98304
Position interpolate from 32x14x14 to 96x32x32
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder.layers.0.self_attn.k_proj.weight', 'decoder.layers.0.self_attn.k_proj.bias', 'decoder.layers.0.self_attn.v_proj.weight', 'decoder.layers.0.self_attn.v_proj.bias', 'decoder.layers.0.self_attn.q_proj.weight', 'decoder.layers.0.self_attn.q_proj.bias', 'decoder.layers.0.self_attn.out_proj.weight', 'decoder.layers.0.self_attn.out_proj.bias', 'decoder.layers.0.self_attn.inner_attn_ln.weight', 'decoder.layers.0.self_attn.inner_attn_ln.bias', 'decoder.layers.0.self_attn_layer_norm.weight', 'decoder.layers.0.self_attn_layer_norm.bias', 'decoder.layers.0.ffn.fc1.weight', 'decoder.layers.0.ffn.fc1.bias', 'decoder.layers.0.ffn.fc2.weight', 'decoder.layers.0.ffn.fc2.bias', 'decoder.layers.0.ffn.ffn_

In [3]:
set(msg.missing_keys)

{'head.bias', 'head.weight'}

In [ ]:
model = model.half()
out = model(torch.randn(1, 1, 384, 512, 512).half())